In [6]:
import json
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tqdm.pandas()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
file = 'result.json'
data = json.load(open(file))
data.keys()

dict_keys(['name', 'type', 'id', 'messages'])

In [3]:
messages = data['messages']
messages[10000]

{'id': 212573,
 'type': 'message',
 'date': '2018-01-22T22:08:41',
 'from': 'Delbar❤ پنکیکی🍩❤️👰🏻',
 'from_id': 4601975800,
 'file': '(File not included. Change data exporting settings to download.)',
 'media_type': 'voice_message',
 'mime_type': 'audio/ogg',
 'duration_seconds': 11,
 'text': ''}

In [27]:
def parse_text(text):
    if isinstance(text, str):
        return text + ' '
    
    all_types = ['bank_card', 'bold', 'code', 'email',
                 'hashtag', 'italic', 'link', 'mention',
                 'phone', 'text_link','underline']
    valid_types = ['bold']
    res = ''
    for t in text:
        if isinstance(t, str):
            res += t + ' '
        elif t['type'] in valid_types:
            res += t['text'] + ' '
    return res

def get_sender(sender):
    return 'soroush' if sender.lower() == 'soroush' else 'erf'

In [23]:
df = pd.DataFrame(data=messages)
df = df[['from', 'text']]
df.head()

,from,text
0,Soroush,
1,Soroush,هدیه ی اولیه تولد! :))
2,Delbar❤ پنکیکی🍩❤️👰🏻,Ey khafan tarin
3,Delbar❤ پنکیکی🍩❤️👰🏻,❤❤❤❤
4,Delbar❤ پنکیکی🍩❤️👰🏻,Zogh haye ziad


In [24]:
df.drop(df[pd.isna(df['from'])].index, inplace=True)
df['from'].unique()

array(['Soroush', 'Delbar❤ پنکیکی🍩❤️👰🏻'], dtype=object)

In [25]:
df['from'] = df.progress_apply(lambda x: get_sender(x['from']), axis=1)
df.head()

,from,text
0,soroush,
1,soroush,هدیه ی اولیه تولد! :))
2,erf,Ey khafan tarin
3,erf,❤❤❤❤
4,erf,Zogh haye ziad


In [28]:
df['text'] = df.progress_apply(lambda x: parse_text(x['text']), axis=1)

In [31]:
df.drop(df[df['text'] == ''].index, inplace=True)
df.shape

(193425, 2)

In [44]:
history = [{'from':df['from'][0] , 'text': df['text'][0]}]

In [45]:
for f,t in tqdm(zip(df['from'][1:], df['text'][1:])):
    if f == history[-1]['from']:
        history[-1]['text'] += ' ' + t
    else:
        history += [{'from':f, 'text': t}]

In [47]:
df2 = pd.DataFrame(data=history)
df2.head(100)

,from,text
0,soroush,هدیه ی اولیه تولد! :))
1,erf,Ey khafan tarin ❤❤❤❤ Zogh haye ziad Melted ...
2,soroush,😌😌😌❤️ wish U enjoy it! :))
3,erf,Kheili khube ha bia berim jaye chaartaar y per...
4,soroush,dar khedmatim! 😏 :)) albatte age shoma eftexa...
...,...,...
95,erf,Mochakeram 🌺 Shab bekheir ! Salam😁 Mishe um...
96,soroush,salam aleykom khanum doctor ettefaghan khasta...
97,erf,Khob avvalan k residan bekheir Dovoman ke bik...
98,soroush,bale bale bande dar khedmate shomam


In [48]:
df2.shape

(103663, 2)

In [49]:
df2.to_csv('processed.csv', index=False)